`model.base_model.model.model` stays unchanged, while `model.base_model.model.lm_head` gets duplicated up to however many heads we want. For each head, apply some mask that will only predict tokens from that head's vocabulary

ohmodel: PeftModelForCausalLM > LoraModel > LlamaForCausalLM > LlamaModel

# MH model construction

In [1]:
%load_ext autoreload
%autoreload 2
from peft import AutoPeftModelForCausalLM, PeftModelForCausalLM, get_peft_config
from transformers import AutoTokenizer, LlamaTokenizer, AutoModelForCausalLM
from multihead_models import MultiHeadPeftModelForCausalLM
import torch
import json
import os

In [2]:


tokenizer = LlamaTokenizer.from_pretrained('/mnt/data/zoo/llama2/llama2-7b-hf/')
inp = tokenizer(
    ['There is a Ubuntu server visible at IP 43.205.13.243, port 22, offering the service cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5.\n',], 
    return_tensors="pt",
)  # Batch size 1
inp = {x:inp[x].cuda(0) for x in inp}

step=60
output_dir = f'/mnt/data/sonia/ckpts/sent3/checkpoint-{step}/'
print(output_dir)
ohmodel = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16)
# outputs = ohmodel.generate(**inp, max_new_tokens=58)
# out=tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False)
# print(out[0])



/mnt/data/sonia/ckpts/sent3/checkpoint-60/


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/mnt/data/sonia/miniconda3/envs/llama/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/mnt/data/sonia/miniconda3/envs/llama/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [3]:
# ohmodel = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16)
mhmodel = MultiHeadPeftModelForCausalLM.from_one_head(ohmodel, [torch.ones(32000)], [1])

In [4]:
inter = mhmodel.generate(**inp, max_length=100)

torch.Size([1, 58, 4096])
torch.Size([1, 58, 32000])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 4096])
torch.Size([1, 1, 32000])
torch.Size([1, 1, 4096])
torc

In [5]:
tokenizer.batch_decode(inter, skip_special_tokens=True, clean_up_tokenization_spaces=False)

['There is a Ubuntu server visible at IP 43.205.13.243, port 22, offering the service cpe:/a:openbsd:openssh:8.2p1 Ubuntu-4ubuntu0.5.\nThe server is running OpenSSH 8.2p1 Ubuntu-4ubuntu0.5.\nThe server is running OpenSSH 8.2p1 Ubuntu-4ubuntu0.5']

In [6]:
len(inp['input_ids'][0])

58

In [7]:
len(inter[0])

100

# "Cloze"

~ special token 3695

In [19]:
inp = tokenizer(
    ['There is a ~ server visible at IP ~, port ~, offering the service ~\n',], 
    return_tensors="pt",
)  # Batch size 1
inp = {x:inp[x].cuda(0) for x in inp}
inp

{'input_ids': tensor([[    1,  1670,   338,   263,  3695,  1923,  7962,   472,  5641,  3695,
          29892,  2011,  3695, 29892, 27032,   278,  2669,  3695,    13]],
        device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
        device='cuda:0')}